In [1]:
import numpy as np
import tensorflow as tf
import os
from random import shuffle
from PIL import Image
#import matplotlib.pyplot as plt

In [2]:
#Directories
s_dir = "/home/vishnu/TFlow_ws/CameraProj/MLData/Smile"
o_dir = "/home/vishnu/TFlow_ws/CameraProj/MLData/Other"
#s_dir = r"C:\Users\Vishnu\Desktop\MLData\Smile"
#o_dir = r"C:\Users\Vishnu\Desktop\MLData\Other"

#Log dir
LOGDIR = "/home/vishnu/TFlow_ws/CameraProj/src/logs/"
#LOGDIR = r"C:\Users\Vishnu\Desktop\MLData\logs"

In [3]:
def get_files(Sdir,Odir):
    filelist = [Sdir+"/"+x for x in os.listdir(Sdir)]+[Odir+"/"+y for y in os.listdir(Odir)]
    shuffle(filelist)
    label = [1 if x.find('Smile') != -1 else 0 for x in filelist]
    return filelist, label
    
def data_aug(filenames,labels):
    img_string = tf.read_file(filenames,name="data_aug_read_files")
    img_decode = tf.image.decode_jpeg(img_string,channels=1)
    img = tf.cast(img_decode,tf.float32,name="cast_float_32")
    img = tf.image.per_image_standardization(img)
    
    mirror_img = tf.image.flip_left_right(img)
    one_hot = tf.one_hot(labels,2)
    
    
    return img,mirror_img,one_hot

def train_data_map(x,y):
    return x,y

In [4]:
#Setting up data for model
filenames,labels = get_files(s_dir,o_dir)
dataset_aug = tf.data.Dataset.from_tensor_slices((filenames,labels))
dataset_aug = dataset_aug.map(data_aug)
#dataset_aug.shuffle()
it = tf.data.Iterator.from_structure(dataset_aug.output_types,dataset_aug.output_shapes)
it_op = it.make_initializer(dataset_aug)


x1,x2,yy = it.get_next()
#ims = tf.stack([x1,x2],0)
#labs = tf.stack([y_,y_],0)
#btc = tf.concat([x1,y_],0)
#Variable initialize operation
init_op = tf.global_variables_initializer()

In [5]:
img_stk = []
label_stk = []
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(it_op)
    
    while(True):
        try:
            r1,r2,r3 = sess.run([x1,x2,yy])
            img_stk.append(r1)
            img_stk.append(r2)
            label_stk.append(r3)
            label_stk.append(r3)
        except tf.errors.OutOfRangeError:
            print("Finished")
            break

print(np.asarray(img_stk).shape)
print(np.asarray(label_stk).shape)

Finished
(6400, 50, 50, 1)
(6400, 2)


In [6]:
#split arrayinto training and other
length = len(img_stk)
t_f = 3*int(length/4.0)

img_stk = np.asarray(img_stk)
label_stk=  np.asarray(label_stk)
Tr_data = img_stk[:t_f]
Tr_data_lab = label_stk[:t_f]
Te_data = img_stk[t_f:]
Te_data_lab = label_stk[t_f:]
print(len(Tr_data))
print(len(Tr_data_lab))
print(len(Te_data))
print(len(Te_data_lab))

4800
4800
1600
1600


In [7]:
#Tflow functions

def conv_layer(m_input,size_in,size_out,k_size_w,k_size_h,pool_k_size,pool_stride_size,name,num):
    with tf.name_scope(name+num):
        w = tf.Variable(tf.truncated_normal([k_size_w,k_size_h,size_in,size_out],stddev = 0.1),name=("w"+num))
        b = tf.Variable(tf.constant(0.1,shape=[size_out]),name=("b"+num))
        
        conv = tf.nn.conv2d(m_input,w,strides=[1,1,1,1],padding="SAME")
        act = tf.nn.leaky_relu((conv+b),alpha=0.3)
        #.3
        
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)
        return tf.nn.max_pool(act,ksize=[1,pool_k_size,pool_k_size,1],strides=[1,pool_stride_size,pool_stride_size,1],padding="SAME")
    
def fc_layer(m_input,size_in,size_out,name,num):
    with tf.name_scope(name+num):
        w = tf.Variable(tf.truncated_normal([size_in,size_out],stddev=0.1),name=("w"+num))
        b = tf.Variable(tf.constant(0.1,shape=[size_out]),name=("b"+num))
        z = tf.matmul(m_input,w)+b
        #bt_norm = batch_norm_fc(z,num)
        #act = tf.nn.leaky_relu(bt_norm,alpha=0.3)
        act = tf.nn.leaky_relu(z,alpha=0.3)
        
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)
        return act

def batch_norm_fc(m_input,num):
    with tf.name_scope("batch_norm"+num):
        beta = tf.Variable(tf.cast(tf.constant(0,shape=[]),tf.float32),name="beta"+num)
        gamma = tf.Variable(tf.cast(tf.constant(1,shape=[]),tf.float32),name="sigma"+num)
        m,var = tf.nn.moments(m_input,axes=[0],keep_dims=True,name="compute_mean_var"+num)
        tf.summary.scalar("beta",beta)
        tf.summary.scalar("gamma",gamma)
        return tf.nn.batch_normalization(m_input,m,var,beta,gamma,variance_epsilon=.05,name="batch_norm"+num)

In [8]:
#model function

def model(start_learning_rate,lr_decay,batch_size,conv_count,fc_count,conv_feats,fc_feats,hparam):
    global LOGDIR
    global Tr_data,Tr_data_lab,Te_data,Te_data_lab
    EPOCHS = float(len(Tr_data))/batch_size
    if (len(conv_feats) != conv_count):
        return
    
    tf.reset_default_graph()
    
    
    with tf.name_scope("place_holder"):
        is_training = tf.placeholder(tf.bool,name="is_training")
        
    with tf.name_scope("data_prep"):
        data_tr = tf.data.Dataset.from_tensor_slices((Tr_data,Tr_data_lab))
        data_tr = data_tr.shuffle(len(Tr_data))
        data_tr = data_tr.map(train_data_map).batch(batch_size)
        
        data_tr_it = tf.data.Iterator.from_structure(data_tr.output_types,data_tr.output_shapes)
        data_tr_it_op = data_tr_it.make_initializer(data_tr)
        
        data_te = tf.data.Dataset.from_tensor_slices((Te_data,Te_data_lab))
        data_te = data_te.shuffle(len(Te_data))
        data_te = data_te.map(train_data_map)
        
        
        data_te_it = tf.data.Iterator.from_structure(data_te.output_types,data_te.output_shapes)
        data_te_it_op = data_te_it.make_initializer(data_te)
        
        def f1():
            x1,y1 = data_tr_it.get_next()
            return x1,y1
        def f2():
            x2,y2 = data_te_it.get_next()
            return x2,y2
        
        x,y = tf.cond(is_training,lambda: data_tr_it.get_next(),lambda: data_te_it.get_next())
        x_image = tf.reshape(x,[-1,50,50,1])
        y_ = tf.reshape(y,[-1,2])
    #with tf.name_scope("place_holders"):
        #x = tf.placeholder(tf.float32,shape=[50,50,1],name="x")
        #x_image = tf.reshape(x,[-1,50,50,1])
        #y = tf.placeholder(tf.float32,shape=[None,2],name="y")
    
    with tf.name_scope("variable"):
        learning_rt = tf.Variable(start_learning_rate,name="learning_rt")
        global_step = tf.Variable(0,trainable=False)
        tf.summary.scalar("Learning_rate",learning_rt)
        
    with tf.name_scope("lr_decay"):
        dec_learning_rate = tf.train.exponential_decay(learning_rt,global_step,EPOCHS,lr_decay,staircase=True)
        
        
    
    #tf.summary.image("image_input",x_image,1)
    
    convs = []
    #convs.append(x_image)
    convs.append(x_image)
    conv_name = "conv"
    for i in range(0,conv_count-1):
        convs.append(conv_layer(convs[i],conv_feats[i],conv_feats[i+1],5,5,2,2,"conv",str(i+1)))
        
    
    shape = (convs[conv_count-1]).get_shape().as_list()
    fc_feats[0] = shape[1]*shape[2]*conv_feats[conv_count-1]
    flatten = tf.reshape(convs[conv_count-1],[-1,fc_feats[0]])
    
    fcs = []
    fcs.append(flatten)
    fcs_name = "FC"
    for i in range(0,fc_count-1):
        fcs.append((fc_layer(fcs[i],fc_feats[i],fc_feats[i+1],"FC",str(i+1))))
    
    
    logts = fcs[len(fcs)-1]

    with tf.name_scope("cross_entropy"):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logts,labels=y_),name="cross_entropy")
        tf.summary.scalar("cross_entropy",cross_entropy)

    with tf.name_scope("Train"):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step = tf.train.AdamOptimizer(dec_learning_rate).minimize(cross_entropy)
    
    with tf.name_scope("accuracy"):
        pred = tf.equal(tf.argmax(logts,1),tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
        tf.summary.scalar("accuracy",accuracy)
    
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR+hparam)
    
    i = 1
    cor = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(data_tr_it_op)

        while(True):
            try:
                a,s = sess.run([train_step,summ],{is_training: True})
                writer.add_summary(s,i)
                i = i+1
            except tf.errors.OutOfRangeError:
                print("Finished")
                break
        
        writer.add_graph(sess.graph)
        writer.close()
        
        sess.run(data_te_it_op)
        
        while(True):
            try:
                cor_pred = sess.run(pred,feed_dict={is_training: False})
                if (cor_pred):
                    cor = cor+1
            except tf.errors.OutOfRangeError:
                print("Finished Val")
                break
        
        print(float(cor))/len(Te_data)

In [9]:
#Feats that worked
#82% lnrt=1e-3 dec = .6
#conv_feats=[1,8,10,12]
#fc_feats=[0,300,150,50,2]
print(len(Te_data))
len_rt= 1e-3
conv_count = 4
fc_count = 5
b_size = [20,30,40,50]
conv_feats=[1,8,10,12]
#fc_feats=[0,300,150,50,2]
fc_feats=[0,300,150,50,2]
for i in b_size:
    model(len_rt,.6,i,conv_count,fc_count,conv_feats,fc_feats,"testb"+str(i))

1600
Finished
Finished Val
0.83875
Finished
Finished Val
0.819375
Finished
Finished Val
0.78375
Finished
Finished Val
0.82
